In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

## TF-IDF 특징 추출
* $TF-IDF(w,d)$
  * 어떤 단어 w가 문서 d 내에서 얼마나 중요한지 나타내는 수치 
  * TF 
    * term frequency 단어가 문서 내에 출현한 횟수
    * 클수록 문서에서 중요한 단어일 것이다
  * IDF
    * inverse document frequency 단어가 출현한 문서의 숫자의 역수
    * DF가 클수록 일반적으로 많이 쓰는 단어일 것이다

In [2]:
# 문서 내에 단어별 빈도수 
def get_tf(document, word_dict=None):
    if word_dict is None:
        word_dict = {}
    words = document.split() # blank 단위
    
    for w in words:
        word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[w]) 
        
    return pd.Series(word_dict).sort_values(ascending=False)

In [3]:
get_tf('안녕 하는 누구야 안녕')

안녕     2
누구야    1
하는     1
dtype: int64

In [4]:
# 단어가 나타난 문서의 개수
def get_df(documents):
    dicts = []
    vocab = set([])
    df = {}
    
    for d in documents:
        tf = get_tf(d)
        dicts += [tf]
        vocab = vocab | set(tf.keys())
    
    for v in list(vocab):
        df[v] = 0
        for dict_d in dicts:
            if dict_d.get(v) is not None:
                df[v] += 1
    
    return pd.Series(df).sort_values(ascending=False)

In [5]:
get_df(['안녕 하는 누구야 안녕 누구야',
        '안녕 하는 민야 안녕',
        '안녕 하는 주야 안녕'])

안녕     3
하는     3
누구야    1
민야     1
주야     1
dtype: int64

In [6]:
def get_tfidf(docs):
    vocab = {}
    tfs = []
    for d in docs:
        vocab = get_tf(d, vocab)
        tfs += [get_tf(d)]
    df = get_df(docs)
    
    from operator import itemgetter
    import numpy as np
    
    stats = []
    for word, freq in vocab.items():
        tfidfs = []
        for idx in range(len(docs)):
            if tfs[idx].get(word) is not None:
                tfidfs += [tfs[idx][word] * np.log(len(docs) / df[word])]
            else:
                tfidfs += [0]
        stats.append((word, freq, *tfidfs, max(tfidfs)))
        
    name_lst = ['doc_'+str(i+1) for i in range(len(docs))]
    return pd.DataFrame(stats, columns=('word',
                                        'freq',
                                        *name_lst,
                                        'max')).sort_values('max', ascending=False)

In [7]:
get_tfidf(['안녕 하는 누구야 안녕 누구야',
        '안녕 하는 민야 안녕',
        '안녕 하는 주야 안녕'])

,word,freq,doc_1,doc_2,doc_3,max
2,누구야,2,2.197225,0.000000,0.000000,2.197225
3,주야,1,0.000000,0.000000,1.098612,1.098612
4,민야,1,0.000000,1.098612,0.000000,1.098612
0,안녕,6,0.000000,0.000000,0.000000,0.000000
1,하는,3,0.000000,0.000000,0.000000,0.000000
